In [ ]:
!pip install python-telegram-bot[passport,http2]
!pip install numpy tensorflow
!pip install opencv-python
!pip install nest-asyncio

In [ ]:
import ssl
import nest_asyncio
import numpy as np
import tensorflow as tf
from telegram import Update, Bot
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
from io import BytesIO
import cv2

nest_asyncio.apply()

TOKEN = "Your Bot Token"

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

class_names = ['Plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

async def start(update: Update, context: CallbackContext):
    await update.message.reply_text('Welcome Welcome How Can i serve You! If You need to train the model write /train if you wanna get help write /help.')

async def help_command(update: Update, context: CallbackContext):
    await update.message.reply_text("""
     /start - Start a conversation with that
     /help - Still in development
     /train - Train The Model
     """)

async def train(update: Update, context: CallbackContext):
    await update.message.reply_text("The Model is training please be patient !")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
    model.save('cifar_classifier.keras')  # Use .keras or .h5 extension
    await update.message.reply_text('Your Bot is ready please send a picture !')

async def handle_message(update: Update, context: CallbackContext):
    if model is not None:
        await update.message.reply_text("Send an Image please !")
    else:
        await update.message.reply_text("Train the model first then send an image !")

async def handle_photo(update: Update, context: CallbackContext):
    file = await context.bot.get_file(update.message.photo[-1].file_id)
    f = BytesIO(await file.download_as_bytearray())
    file_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    prediction = model.predict(np.array([img / 255.0]))
    await update.message.reply_text(f"In this image i see {class_names[np.argmax(prediction)]}")

def main():
    application = Application.builder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(CommandHandler("train", train))
    application.add_handler(MessageHandler(filters.PHOTO, handle_photo))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    application.run_polling()

# In Colab, use this to prevent the event loop issue
import asyncio
asyncio.run(main())
